In [1]:
import os
import spacy
#
import pandas as pd
#
from spacy.matcher import Matcher
#
from textblob_de import TextBlobDE as TextBlob

In [2]:
nlp = spacy.load('de_core_news_md')
pd.set_option('display.max_colwidth', None)
#os.environ["SPACY_WARNING_IGNORE"] = "W008"

In [3]:
REVIEWS_CLEANED_UNLABELED_FILE_PATH = 'data/reviews_merged_unlabeled_cleaned.csv'
REVIEWS_CLEANED_FILE_PATH = 'data/reviews_merged_cleaned.csv'

df = pd.read_csv(REVIEWS_CLEANED_UNLABELED_FILE_PATH, sep=';')
df.head()

,Unnamed: 0,caption,rating
0,0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch,4.0
1,1,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter,5.0
2,2,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult,5.0
3,3,gestern dritt selben burger bestellt gestern abend magen darm burger lecker irgendetwas schmeckte komisch erwartet,1.0
4,4,fanden burger restaurant klasse individuelle möglichkeiten zudem fleischlose alternativen ambiente zeitsprung vergangenheit kartenzahlung heutzutage zeitgemäß,4.0


In [4]:
categories = ['essen', 'service', 'atmosphäre', 'preis', 'warten']
categories_columns = ['food_positive', 'food_negative',
                        'service_positive', 'service_negative',
                        'ambient_positive', 'ambient_negative',
                        'price_positive', 'price_negative',
                        'waiting_positive', 'waiting_negative']

In [5]:
def spacy_most_similar(word, topn=10):
    ms = nlp.vocab.vectors.most_similar(
        nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [6]:
spacy_most_similar(categories[0])

(['Aufessen',
  'weggefuttert',
  'drinken',
  'einzukochen',
  'koten',
  'inhaliere',
  'knabbere',
  'Bio-Essen',
  'schmatz',
  'Trinket'],
 array([[1.    , 0.8243, 0.8131, 0.7734, 0.7115, 0.7059, 0.7032, 0.6845,
         0.672 , 0.6681]], dtype=float32))

### Aspect based sentiment analysis

In [7]:
matcher = Matcher(nlp.vocab)
patterns = [
    [{'POS':'ADJ', 'OP': '+'}, {'POS':'NOUN'}],
    [{'POS':'NOUN'}, {'POS':'VERB'}, {'POS':'ADJ'}]
]
matcher.add("category", patterns)

df_matches = pd.DataFrame([], columns=['index', 'aspect', 'entity_group'])

similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption']):
    doc = nlp(wordText)
    matches = matcher(doc)

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]

        for i, s in enumerate(span):
            if s.pos_ in ['NOUN']:
                category = ''
                similarity = 0

                similarity_cat0 = nlp(categories[0]).similarity(span)
                if similarity_cat0 > similarity_min: # fits into
                    category = categories[0]
                    similarity = similarity_cat0

                print(span)
                similarity_cat1 = nlp(categories[1]).similarity(span)
                if similarity_cat1 > similarity_min: # fits into
                    category = categories[1]
                    similarity = similarity_cat1

                similarity_cat2 = nlp(categories[2]).similarity(span)
                if similarity_cat2 > similarity_min: # fits into
                    category = categories[2]
                    similarity = similarity_cat2

                similarity_cat3 = nlp(categories[3]).similarity(span)
                if similarity_cat3 > similarity_min: # fits into
                    category = categories[3]
                    similarity = similarity_cat3

                similarity_cat4 = nlp(categories[4]).similarity(span)
                if similarity_cat4 > similarity_min: # fits into
                    category = categories[4]
                    similarity = similarity_cat4

                df_matches = pd.concat([pd.DataFrame({'index': rowIndex,
                                                      'aspect': category,
                                                      'entity_group': span.text,
                                                      }, columns=df_matches.columns, index=[0]),
                                        df_matches])

ambiente konzept
cooles ambiente konzept
lecker bürger
individuelle bedürfnisse
sympathische mitarbeiter
gemütliche atmosphäre
freundliche gemütliche atmosphäre
fantastisches essen
burger schicki-micki
absoluter kult
burger lupo
selben burger
individuelle möglichkeiten
ambiente zeitsprung
alternativen ambiente zeitsprung
fleischlose alternativen ambiente zeitsprung
bester service
lecker burger
fairen preis
ambiente kartenzahlung
coolen ambiente kartenzahlung
leckere burger
freundliche bedienung
klasse auswahl
milchshake kombination
sterne milchshake kombination
veganer vegetarier
bescheidenen services
kreative auswahl
schlechten erfahrungen
tolle atmosphäre
leckere burger
leckere burger
nette bedienung
burger d-dorf
handgemachte burger
lecker gast
tische minuten
kalter burger
nettes personal
erstklassige burger
witziges ambiente
lieblingslokale stadt
eingerichtetes restaurant
lockeren servicemitarbeitern
größer sparmenüs
versteckte kamera
cooles ambiente
fairen preisen
bester burger
ba

/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat0 = nlp(categories[0]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:36: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat2 = nlp(categories[2]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:41: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat3 = nlp(categories[3]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:46: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarit

schlechteste döner
peperoni döner
beste peperoni döner
döner preis
einziger stern
döner schmeckt absoluter
absoluter standard
appetit gesehen anderer
anderer zutaten
negative emotionen
qualität verschlechtert rote
soße geschmack
rote soße geschmack
restaurant gerannt letztes
letztes döner
negativen sinne
unfreundliche bedienung
soße ertränkt
monatlichen preiserhöhungen
look kind
erwartete döner
teurer preis
besseren döner
unverhältnismäßiger preisanstieg
döner plastik
durchschnittlicher standard
günstigste preis
besonderes fett
leidenschaftlicher essenkritiker
totzdessen gemüse
döner totzdessen gemüse
soße brot
richtiges fleisch
schlechte qualität
döner stadt
beste döner stadt
soße soße
ähnlichem sterne
netten mitarbeiter
letzter gefühl
düsseldorfer innenstadt
bessere gegessen
update chance
lustiges personal
herzliches lustiges personal
lieblingsdöner herzliches lustiges personal
absoluter lieblingsdöner herzliches lustiges personal
bester döner
döner geschmack
bester döner geschmack
ü

/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat0 = nlp(categories[0]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:36: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat2 = nlp(categories[2]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:41: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat3 = nlp(categories[3]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_55707/2380499998.py:46: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarit

leckere pizza
vegane käse
lecker vegane käse
pizza essen getränke
getränke theke
pizza niveau
zuvorkommendes personal
freundliches zuvorkommendes personal
schönen abend
neapolitanische pizza
vernünftige neapolitanische pizza
beste pizza
italiens stimmung
geschmackssache pizza
nettes personal
angenehmes ambiente
neapolitanische pizza
hervorragende neapolitanische pizza
fruendliche atmosphäre
schöne auswahl
leckere pizza
jüngere volk


In [8]:
df_matches

,index,aspect,entity_group
0,332,,jüngere volk
0,330,essen,leckere pizza
0,329,,schöne auswahl
0,329,atmosphäre,fruendliche atmosphäre
0,328,,hervorragende neapolitanische pizza
...,...,...,...
0,1,atmosphäre,sympathische mitarbeiter
0,0,,individuelle bedürfnisse
0,0,essen,lecker bürger
0,0,atmosphäre,cooles ambiente konzept


#### TextBlob sentiment analysis

In [9]:
sentiment_polarities = []
sentiment_subjectivities = []

for i, row in df_matches.iterrows():
    sentiment = TextBlob(row['entity_group']).sentiment
    sentiment_polarities.append(sentiment.polarity)
    sentiment_subjectivities.append(sentiment.subjectivity)

df_matches['sentiment_polarity'] = sentiment_polarities
df_matches['sentiment_subjectivity'] = sentiment_subjectivities

df_matches_sorted = df_matches.sort_values(by='index')
print(df_matches_sorted)

   index      aspect                         entity_group  sentiment_polarity  \
0      0  atmosphäre                     ambiente konzept                 0.0   
0      0                         individuelle bedürfnisse                 0.0   
0      0  atmosphäre              cooles ambiente konzept                 0.7   
0      0       essen                        lecker bürger                 0.0   
0      1       essen                  fantastisches essen                 1.0   
..   ...         ...                                  ...                 ...   
0    328              hervorragende neapolitanische pizza                 1.0   
0    329  atmosphäre               fruendliche atmosphäre                 0.0   
0    329                                   schöne auswahl                 1.0   
0    330       essen                        leckere pizza                 0.0   
0    332                                     jüngere volk                 0.0   

    sentiment_subjectivity 

In [10]:
df.head(3)

,Unnamed: 0,caption,rating
0,0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch,4.0
1,1,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter,5.0
2,2,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult,5.0


In [11]:
# Loop through all df_matches
# For each index check the aspect, if it matches a category
# Update the right category of df with the respective sentiment
for column in categories_columns:
    df[column] = 0

for rowIndex, row in df.iterrows():
    entities_by_index = df_matches_sorted.loc[df_matches_sorted['index'] == rowIndex]
    # reset
    for j, item in entities_by_index.iterrows():
        if item['aspect'] != '':
            category_i = categories.index(item['aspect'])
            polarity = item['sentiment_polarity']
            if polarity != 0:
                column_index = (category_i * 2) if polarity > 0 else (category_i * 2) + 1
                df.at[rowIndex, categories_columns[column_index]] = 1

df.head()

,Unnamed: 0,caption,rating,food_positive,food_negative,service_positive,service_negative,ambient_positive,ambient_negative,price_positive,price_negative,waiting_positive,waiting_negative
0,0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch,4.0,0,0,0,0,1,0,0,0,0,0
1,1,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter,5.0,1,0,0,0,1,0,0,0,0,0
2,2,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult,5.0,0,0,0,0,0,0,0,0,0,0
3,3,gestern dritt selben burger bestellt gestern abend magen darm burger lecker irgendetwas schmeckte komisch erwartet,1.0,0,0,0,0,0,0,0,0,0,0
4,4,fanden burger restaurant klasse individuelle möglichkeiten zudem fleischlose alternativen ambiente zeitsprung vergangenheit kartenzahlung heutzutage zeitgemäß,4.0,0,0,0,0,0,0,0,0,0,0


In [12]:
df.to_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')